# Antennas

This notebook demonstrates the plotting capabilities of Ephemerista's antenna code:

* Gain pattern, both in linear and polar plots
* Contour plots on a map
* Visibility cone in 3D 

Notes:

* For the polar plots the $\theta$ angle (i.e. corresponding to the elevation) is in the [0, 360°] range. This is only for the polar plots, in reality the $\theta$ angle is in the [0, 180°] range, or [-90, 90°] or in the [0, 90°] depending on the $\theta$/$\phi$ conventions for parametrizing antenna patterns.
* As of now, all antennas depend only on the $\theta$ angle and are $\phi$-invariant. However, this might change in the future when new antenna types are introduced.

In [ ]:
import ephemerista

ephemerista.init(eop_path="../tests/resources/finals2000A.all.csv", spk_path="../tests/resources/de440s.bsp")

In [ ]:
from ephemerista.propagators.sgp4 import SGP4

iss_tle = """ISS (ZARYA)
1 25544U 98067A   24187.33936543 -.00002171  00000+0 -30369-4 0  9995
2 25544  51.6384 225.3932 0010337  32.2603  75.0138 15.49573527461367"""

iss_prop = SGP4(tle=iss_tle)
c_iss = iss_prop.propagate(iss_prop.time)

geo_tle = """EUTELSAT 36D
1 59346U 24059A   24317.58330181  .00000165  00000+0  00000+0 0  9993
2 59346   0.0146 287.8797 0000094 214.1662 156.0172  1.00269444  2311"""
geo_prop = SGP4(tle=geo_tle)
c_geo = geo_prop.propagate(geo_prop.time)

In [ ]:
import plotly.graph_objects as go

## Parabolic antenna

We define a parabolic antenna of 75cm diameter, 60% efficiency. We first use this antenna at 8.4 GHz (X band).

In [ ]:
from ephemerista.comms.frequencies import Frequency

freq_parabol = Frequency.gigahertz(8.4)

In [ ]:
from ephemerista.comms.antennas import ComplexAntenna, ParabolicPattern

In [ ]:
parabolic_antenna = ComplexAntenna(pattern=ParabolicPattern(diameter=0.75, efficiency=0.6))

### Polar pattern diagram

In [ ]:
fig = go.Figure()

fig.add_trace(parabolic_antenna.plot_pattern(frequency=freq_parabol))

fig.update_layout(
    title="Antenna pattern diagram (polar) [dBi]", polar={"angularaxis": {"rotation": 90, "direction": "clockwise"}}
)
fig.show()

### Linear pattern diagram

In [ ]:
fig = go.Figure()
fig.add_trace(parabolic_antenna.plot_pattern(frequency=freq_parabol, fig_style="linear"))
fig.update_layout(title="Antenna pattern diagram (cartesian) [dBi]", xaxis_range=[-90.0, 90.0])
fig.show()

### Contour map plots

#### ISS orbit, X band

In [ ]:
geomap = parabolic_antenna.plot_contour_2d(frequency=freq_parabol, sc_state=c_iss)
display(geomap)

#### ISS orbit, Ka band

In [ ]:
geomap = parabolic_antenna.plot_contour_2d(frequency=Frequency.gigahertz(31), sc_state=c_iss)
display(geomap)

#### Geostationary satellite, Ka band

In [ ]:
geomap = parabolic_antenna.plot_contour_2d(frequency=Frequency.gigahertz(31), sc_state=c_geo)
display(geomap)

## Dipole antennas

In [ ]:
freq_dipole = Frequency.megahertz(433)

In [ ]:
from ephemerista.comms.antennas import DipolePattern
from ephemerista.comms.utils import wavelength

### Half wavelength vs short dipole: polar pattern diagrams

In [ ]:
dipole_halfwavelength = ComplexAntenna(
    pattern=DipolePattern(length=wavelength(frequency=freq_dipole.hertz) / 2), boresight_vector=[0, 1, 0]
)
dipole_short = ComplexAntenna(pattern=DipolePattern(length=wavelength(frequency=freq_dipole.hertz) / 1000))

In [ ]:
fig = go.Figure()
fig.add_trace(dipole_halfwavelength.plot_pattern(frequency=freq_dipole, trace_name="Half wavelength"))
fig.add_trace(dipole_short.plot_pattern(frequency=freq_dipole, trace_name="Short dipole"))

fig.update_layout(
    title="Antenna pattern diagram (polar) [dBi]",
    polar={"radialaxis": {"range": [-12.0, 3.0]}, "angularaxis": {"rotation": 90, "direction": "clockwise"}},
)
fig.show()

Now plotting the gains relative to the peak gain (so lower than 0 dB), to compare with https://en.wikipedia.org/wiki/Dipole_antenna#/media/File:L-over2-rad-pat.svg

In [ ]:
fig = go.Figure()
fig.add_trace(
    dipole_halfwavelength.plot_pattern(frequency=freq_dipole, trace_name="Half wavelength", relative_to_peak=True)
)
fig.add_trace(dipole_short.plot_pattern(frequency=freq_dipole, trace_name="Short dipole", relative_to_peak=True))

fig.update_layout(
    title="Antenna pattern diagram (polar, gain relative to peak gain) [dBd]",
    polar={"radialaxis": {"range": [-12.0, 2]}, "angularaxis": {"rotation": 90, "direction": "clockwise"}},
)
fig.show()

### Contour map plot, ISS orbit

Unlike the parabolic antenna, we mount the dipole antenna in the horizontal plane (boresight vector [0, 1, 0] in VVLH frame) so that the gain in the nadir direction is maximal.

In [ ]:
dipole_halfwavelength.plot_contour_2d(frequency=freq_dipole, sc_state=c_iss)

### 3/2 wavelength

In [ ]:
fig = go.Figure()
fig.add_trace(
    ComplexAntenna(pattern=DipolePattern(length=3 * wavelength(frequency=freq_dipole.hertz) / 2)).plot_pattern(
        frequency=freq_dipole
    )
)

fig.update_layout(
    title="Antenna pattern diagram (polar) [dBi]",
    polar={"radialaxis": {"range": [-20.0, 5.0]}, "angularaxis": {"rotation": 90, "direction": "clockwise"}},
)
fig.show()

### 5/4 wavelength

In [ ]:
fig = go.Figure()
fig.add_trace(
    ComplexAntenna(pattern=DipolePattern(length=5 * wavelength(frequency=freq_dipole.hertz) / 4)).plot_pattern(
        frequency=freq_dipole
    )
)

fig.update_layout(
    title="Antenna pattern diagram (polar) [dBi]",
    polar={"radialaxis": {"range": [-20.0, 6.0]}, "angularaxis": {"rotation": 90, "direction": "clockwise"}},
)
fig.show()

## 3D visibility cone

The following cell shows 2 visibility cones corresponding to the following frequencies and satellites:

* A parabolic antenna at 8.4 GHz from a geostationary satellite, pointed towards nadir
* A parabolic antenna at 2.2 GHz from the ISS, pointed towards the velocity vector

In [ ]:
from ephemerista.plot.utils import ensure_3d_cube_aspect_ratio

fig = go.Figure()

planet_mesh3d = c_geo.origin.plot_3d_surface()
fig.add_trace(planet_mesh3d)

cone_viz_geo = parabolic_antenna.viz_cone_3d(
    frequency=Frequency.gigahertz(8.4), sc_state=c_geo, opacity=0.5, name="GEO sat"
)
fig.add_trace(cone_viz_geo)
fig.add_trace(
    go.Scatter3d(x=[c_geo.position[0]], y=[c_geo.position[1]], z=[c_geo.position[2]], name="GEO sat", mode="markers")
)

parabol_towards_velocity = ComplexAntenna(
    pattern=ParabolicPattern(diameter=0.75, efficiency=0.6), boresight_vector=[1, 0, 0]
)
cone_viz_iss = parabol_towards_velocity.viz_cone_3d(
    frequency=Frequency.gigahertz(2.2), sc_state=c_iss, opacity=0.5, name="ISS", cone_length=20000
)
fig.add_trace(cone_viz_iss)
fig.add_trace(
    go.Scatter3d(x=[c_iss.position[0]], y=[c_iss.position[1]], z=[c_iss.position[2]], name="ISS", mode="markers")
)

fig.update_layout(
    title="Antenna cone 3D visualization",
    autosize=False,
    width=1000,
    height=700,
)

ensure_3d_cube_aspect_ratio(fig)

fig.show()

### MSI Import

In [ ]:
from pathlib import Path

from ephemerista.comms.antennas import MSIPattern

msi_file = Path("..") / "tests" / "resources" / "antennas" / "cylindricalDipole.pln"
pattern = MSIPattern.read_file(msi_file)
frequency = pattern.frequency
antenna = ComplexAntenna(pattern=pattern)

fig = go.Figure()
fig.add_trace(ComplexAntenna(pattern=pattern).plot_pattern(frequency=frequency, trace_name="MATLAB"))
fig.add_trace(
    ComplexAntenna(pattern=DipolePattern(length=2.0)).plot_pattern(frequency=frequency, trace_name="Ephemerista")
)

fig.update_layout(
    title="Antenna pattern diagram (polar) [dBi]",
    polar={"radialaxis": {"range": [-20.0, 6.0]}, "angularaxis": {"rotation": 90, "direction": "clockwise"}},
)
fig.show()